In [1]:
# imports
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import dill as pickle
from datetime import datetime
from writer import date_stamp, time_stamp
import collections
import numpy as np
import string
import re

In [2]:
# set driver to chrome
driver = webdriver.Chrome("/Applications/chromedriver")

### Scrape Setlists

In [3]:
# go to the umphreys all things URL
url = "https://allthings.umphreys.com/setlists/"
driver.get(url)

In [4]:
# load the main page
content = driver.page_source
soup = BeautifulSoup(content)

In [5]:
shows = []

# iterate through the years
for year in soup.findAll('a', attrs={'class':'setlistbutton btn btn-small btn-default'}):
    year_url = url + year['rel'][0] + ".html"
    # go to year url
    driver.get(year_url)
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    # scrape every show of this year
    for show in soup.findAll('section', attrs={'class':'setlist'}):
        
        # get show date and title of show
        header = show.find('h3', attrs={'class':'splashtitle'})
        date = header.find('a', attrs={'class':'setlistdate'}).findAll(text=True)[0].strip()
        title = ''.join(header.findAll(text=True)[1:]).strip()
        
        # Ignore Brendan and Jake shows
        if 'Brendan and Jake' in title:
            continue
        
        # get dictionary of setlabel to setlist for every set
        if 'Setlist Unknown (but it was probably awesome)' in str(show.findAll('p',recursive=False, text=True)):
            sets = None
        else:
            sets = {}
            for setlist in show.findAll('p',recursive=False):
                setlabel = ''.join(setlist.find('b', attrs={'class':'setlabel'}).findAll(text=True))
                setlabel = setlabel.strip().rstrip(':')
                
                # Create useful objects representing the setlist
                setlist_str = setlist.findAll(text=True)
                set_label_parts = 3 if setlabel in ['1st Encore', '2nd Encore', '3rd Encore'] else 1
                songs_transitions_tags = [x.strip() for x in setlist_str[set_label_parts:] if x.strip() != '']
                
                # Manually add missing transitions on this date
                if date == '02.21.2004':
                    i = songs_transitions_tags.index('All In Time')
                    songs_transitions_tags[i:i+1] = ['All In Time', ',']
                    i = songs_transitions_tags.index('Waiting Room')
                    songs_transitions_tags[i:i+1] = [',', 'Waiting Room']                    
                 
                # Extract transitions from unlinked to songs
                i = 0
                while i < len(songs_transitions_tags):
                    x = songs_transitions_tags[i]
                    x_split = x.split(' ')
                    if len(x_split) > 1:
                        out_trans = x_split[-1]
                        if out_trans in [',', '>', '->']:
                            songs_transitions_tags[i:i+1] = [x.rstrip(out_trans).strip(), out_trans]
                            continue
                        in_trans = x_split[0]
                        if in_trans in [',', '>', '->']:
                            songs_transitions_tags[i:i+1] = [in_trans, x.lstrip(in_trans).strip()]
                            continue     
                    if len(x) > 1 and x[-1] == ',':
                        songs_transitions_tags[i:i+1] = [x.rstrip(',').strip(), ',']
                        continue
                    i += 1
                    
                # Manual change
                try:
                    i = songs_transitions_tags.index('unknown song,  unknown song')
                    songs_transitions_tags[i:i+1] = ['unknown song', ',', 'unknown song']
                except:
                    pass
                
                # Remove tags from setlist object maintaining the index of the corresponding song
                i = 0
                tags = []
                songs_transitions = songs_transitions_tags.copy()
                while i < len(songs_transitions):
                    if re.match(r"([[]\d+[]])", songs_transitions[i]):
                        tag_num = int(songs_transitions[i].strip('[]'))
                        prev_index = i-1
                        tags.append((tag_num, prev_index))
                        songs_transitions.pop(i)            
                    else:
                        i += 1
                        
                # Create blank dictionary of songs
                songs = []
                for i in np.arange(0,len(songs_transitions),2):
                    songs.append({'name' : songs_transitions[i],
                                  'stewart' : None, 
                                  'in_transition' : None,
                                  'out_transition' : None,
                                  'tag' : None})
                    
                def offset(a, b):
                    off = 0
                    ret = []
                    a_i = 0
                    b_i = 0
                    while b_i < len(b):
                        if a[a_i] == b[b_i]:
                            a_i += 1
                            b_i += 1
                            ret.append(off)
                        else:
                            a_i += 1
                            off += 1
                    return ret
                
                # Get Jimmy Stewarts for each song; update dictionary
                linked_songs = [x.findAll(text=True)[0].strip() for x in setlist.findAll('a', recursive=False)]
                all_songs = [x['name'] for x in songs]
                if len(linked_songs) > 0:
                    stewart_offset = offset(all_songs, linked_songs)
                    linked_songs_elements = setlist.findAll('a', recursive=False)
                    for i in range(len(linked_songs_elements)):
                        try:
                            stewart = '_'.join(linked_songs_elements[i]['class'])
                        except KeyError:
                            stewart = None
                        songs[i - stewart_offset[i]]['stewart'] = stewart
                           
                # Get tags for each song; update dictionary              
                for tag in tags:
                    song_index = int(tag[1]/2)
                    songs[song_index]['tag'] = tag[0]
                
                # Get transitions for each song; update dictionary  
                for i in np.arange(1,len(songs_transitions),2):
                    out_song_index = int((i-1)/2)
                    songs[out_song_index]['out_transition'] = songs_transitions[i]
                    in_song_index = int((i+1)/2)
                    songs[in_song_index]['in_transition'] = songs_transitions[i]
                
                sets[setlabel] = songs
        
        # get setlist meta date
        meta = ''.join(show.find('div', attrs={'class':'showmeta'}).findAll(text=True))
        
        # get the reviews (if any)
        tmp = meta.split('Reviews:')
        reviews = None if len(tmp) == 1 else tmp[1].strip()
        meta = tmp[0].strip()
        
        # remove the purchase via UMLive info (also removes reviews)
        meta = meta.split('Purchase via UMLive')[0]
        
        # get the support (if any)
        tmp = meta.split('Support:')
        support = None if len(tmp) == 1 else tmp[1].strip(' \n')
        meta = tmp[0].strip()
        
        # get the notes (if any)
        tmp = meta.split('Notes:')
        notes = None if len(tmp) == 1 else tmp[1].split('\n')
        meta = tmp[0].strip()
        
        # get the tag captions
        tags = re.split('([[]\d+[]].)',meta)[1:]
        tmp = {}
        for i in range(int(len(tags)/2)):
            tmp[int(tags[2*i].strip(' ]['))] = tags[2*i+1]
        tags = tmp 
        
        vip = False
        if notes is not None and 'VIP' in '\n'.join(notes):
            vip = True
        
        # add show to shows
        shows.append(dict(date=date,
                          title=title,
                          vip=vip,
                          sets=sets,
                          reviews=reviews,
                          support=support,
                          notes=notes,
                          tags=tags))

### Format Data

In [6]:
# Manually add country to these two shows
shows[[i for i in range(len(shows)) if shows[i]['date'] == '02.12.2020'][0]]['title'] = (
    'Graceland Soundstage at Elvis Presley’s Memphis, Memphis, TN, USA')
shows[[i for i in range(len(shows)) if shows[i]['date'] == '02.13.2020'][0]]['title'] = (
    'The Caverns, Pelham, TN, USA')

In [7]:
extracted_shows = []
extracted_songs = []
for show in shows:
    # get all data for shows
    date = show['date'].replace('.','-')
    title = show['title']
    country = title.split(', ')[-1]
    if country in ['Australia', 'Canada', 'UK', 'USA']:
        state = title.split(', ')[-2]
        city = title.split(', ')[-3]
        venue = ', '.join(title.split(', ')[:-3])
    else:
        state = None
        city = title.split(', ')[-2]
        venue = ', '.join(title.split(', ')[:-2]).strip('"')
    vip = show['vip']
    set_ct = -1 if show['sets'] is None else len(show['sets'])
    song_ct = -1 if set_ct == -1 else sum([len(setlist) for setlist in show['sets'].values()])
    support = show['support']
    notes = show['notes']
    tags = show['tags']
    
    city_str = city.replace(' ','_')
    counts_list = [str(set_ct),str(song_ct)] if set_ct != -1 else ['UNK']
    
    # create show primary keys
    if vip:
        primary_key = '-'.join([date,city_str,'vip'] + counts_list)
    else:
        primary_key = '-'.join([date,city_str] + counts_list)
        
    # add this show to list of extracted shows
    extracted_shows.append({'primary_key' : primary_key,
                            'date' : date,  
                            'title' : title, 
                            'venue' : venue,
                            'city'  : city,
                            'state' : state,
                            'country' : country,
                            'vip' : vip, 
                            'support' : support,
                            'notes' : notes,
                            'tags' : tags,
                            'set_ct' : set_ct, 
                            'song_ct' : song_ct})
    
    # iterate through songs played at this show
    setlists = show['sets']
    if setlists is not None:
        already_played = set() # maintain songs already played at this show
        for setlist in setlists:
            set_name = setlist
            songs = setlists[setlist]
            for i in range(len(songs)):
                song_name = songs[i]['name']
                parent = song_name not in already_played
                already_played.add(song_name)
                # append to extracted songs
                extracted_songs.append({**setlists[setlist][i], **{'show' : primary_key,
                                                                   'set' : set_name,
                                                                   'order' : i+1,
                                                                   'parent' : parent}})

# convert to pandas dataframes
extracted_shows = pd.DataFrame(extracted_shows)
extracted_songs = pd.DataFrame(extracted_songs)

# ensure primary keys are unique
assert len(extracted_shows[extracted_shows['primary_key'].duplicated()]) == 0
extracted_shows = extracted_shows.set_index('primary_key')

# clean songs dataframe
extracted_songs['stewart_with_lyrics'] = extracted_songs.stewart == 'stewart_withlyrics'
extracted_songs['stewart'] = extracted_songs.stewart.isin(['stewart', 'stewart_withlyrics'])
extracted_songs['tag'] = extracted_songs['tag'].fillna(-1).astype(int)
extracted_songs = extracted_songs[['name', 'show', 'set', 'order', 'parent', 
                                   'out_transition', 'tag', 'stewart', 'stewart_with_lyrics']]

# rename songs that are not in aggregate statistics
extracted_songs['name'] = extracted_songs['name'].replace('"Jimmy Stewart"', 'Jimmy Stewart')
extracted_songs['name'] = extracted_songs['name'].replace('Improvisation', 'Jimmy Stewart')
extracted_songs['name'] = extracted_songs['name'].replace('Drum solo', 'Solo - Drums')
extracted_songs['name'] = extracted_songs['name'].replace('drum solo', 'Solo - Drums')
extracted_songs['name'] = extracted_songs['name'].replace('piano improv', 'Solo - Piano')
extracted_songs['name'] = extracted_songs['name'].replace('piano solo', 'Solo - Piano')
extracted_songs['name'] = extracted_songs['name'].replace('"Zappa (Jake conducts) improv"', 'Zappa Jimmy Stewart')
extracted_songs['name'] = extracted_songs['name'].replace('OM Trio Jam', 'Jam w/ OM Trio')
extracted_songs['name'] = extracted_songs['name'].replace('Percussion', 'Solo - Percussion')
extracted_songs['name'] = extracted_songs['name'].replace('bass jam', 'Solo - Bass')
extracted_songs['name'] = extracted_songs['name'].replace('unknown song', 'UNK')
extracted_songs['name'] = extracted_songs['name'].replace('"Get Weird" JO', 'Jam - Get Weird')
extracted_songs['name'] = extracted_songs['name'].replace('"Horns vs. Band Challenge"', 'Horns vs. Band Challenge')
extracted_songs['stewart'] = (extracted_songs['stewart'] | 
                              extracted_songs['name'].isin(['Jimmy Stewart', 'Zappa Jimmy Stewart']))

### Scrape Aggregated Songs

In [8]:
# go to the umphreys all things URL
url = "https://allthings.umphreys.com/song/"
driver.get(url)

In [9]:
# load the main page
content = driver.page_source
soup = BeautifulSoup(content)

In [10]:
songs = {}
table = soup.find('table', attrs={'id' : 'songtable'})
for row in table.findAll('tr')[2:]:
    tmp = row.findAll('td')
    if tmp[0].findAll(text=True)[0] in ['Happy Birthday', 'XRT Xmas Promo Medley']:
        songs['Happy Birthday'] = {'artist' : 'UKN',
                                   'times_played' : int(tmp[2].findAll(text=True)[0]),
                                   'debut_date' : tmp[3].findAll(text=True)[0].replace('-','.'),
                                   'last_played' : tmp[4].findAll(text=True)[0].replace('-','.')}
    else:
        cells = [cell.findAll(text=True)[0].strip() for cell in tmp]
        songs[cells[0]] = {'artist' : cells[1],
                           'times_played' : int(cells[2]),
                           'debut_date' : cells[3].replace('-','.'),
                           'last_played' : cells[4].replace('-','.')}

In [11]:
# create CSV object
songs = pd.DataFrame(songs).transpose()

# deal with song titles that were too long
setlist_songs = []
for show in shows:
    setlists = show['sets']
    if setlists is not None:
        for setlist in setlists:
            for song in setlists[setlist]:
                setlist_songs.append(song['name'])
setlist_songs = list(set(setlist_songs))

too_long_dict = {}
for too_long in [x for x in list(songs.index) if x[-3:] == '...']:
    too_long_dict[too_long] = [x for x in setlist_songs if too_long[:32] in x][0]

songs = songs.rename(index=too_long_dict)
display(songs.head())

,artist,times_played,debut_date,last_played
...And Justice for All,Metallica,28,2003.10.31,2020.03.06
...And We Became Sunshine,The Uglysuit,1,2010.09.25,2010.09.25
"""Brendan & Jake switch rigs""",Umphrey's McGee,2,2010.04.24,2014.05.03
"""Mrs Robinson's Strut""",Umphrey's McGee,1,2011.04.02,2011.04.02
(Don't Fear) The Reaper,Blue Öyster Cult,14,2001.10.31,2013.08.15


### Validate Data

In [12]:
# check aganist aggregate statistics
songs['extract_count'] = extracted_songs[extracted_songs.parent == True].groupby('name').count()['parent']
assert(len(songs[(songs['extract_count'] - songs['times_played']) > 0]) < 3)
display(songs[(songs['extract_count'] - songs['times_played']) > 0])

# songs in setlist but not listed in stats
set_list_songs = list(extracted_songs.name.drop_duplicates())
missing = []
for x in set_list_songs:
    if x not in list(songs.index):
        missing.append(x)
missing = list(set(missing))
print('In setlists:', missing)

# songs in stats but not listed in setlist 
missing = []
for x in list(songs.index):
    if x not in set_list_songs:
        missing.append(x)
missing = list(set(missing))
assert(len(missing) == 0)

,artist,times_played,debut_date,last_played,extract_count
G-Song,Umphrey's McGee,81,1998.01.29,2014.11.08,82
Jam,Umphrey's McGee,50,1998.11.04,2019.06.22,51


In setlists: ['Solo - Bass', 'Jamiroquai #7', 'Hair Band Medley', 'Jimmy Stewart', 'Horns vs. Band Challenge', 'Jazz Odyssey', 'Medley', 'Halloween Medley', 'Irish Jig', 'Zappa Jimmy Stewart', 'Solo - Drums', 'Solo - Piano', 'UNK', 'Jam - Get Weird', 'Jam w/ OM Trio', 'XRT Xmas Promo Medley', 'Solo - Percussion']


In [13]:
# Add artist to extracted songs
extracted_songs = extracted_songs.merge(songs['artist'].reset_index()
                                                       .rename(columns={'index' : 'name'}), 
                                        on='name', how='left')
extracted_songs['artist'] = extracted_songs['artist'].fillna("Umphrey's McGee")
extracted_songs = extracted_songs[['name', 'artist', 'show', 'set', 'order', 'parent', 
                                   'out_transition', 'tag', 'stewart', 'stewart_with_lyrics']]

### Song Abbreviations

In [14]:
shows = extracted_shows
songs = extracted_songs

In [15]:
artist = songs[['name','artist']].drop_duplicates().set_index('name')['artist'].to_dict()

In [16]:
song_to_abbrv = {}
abbrv_to_song = {}
f = open("data/song_codes.txt", "r")
for i in f:
    name = i.split('= ')[0]
    code = i.split('= ')[1].strip('\n ')
    code = code.upper() if artist[name] == "Umphrey's McGee" else code.lower()
    song_to_abbrv[name] = code
    abbrv_to_song[code] = name

In [17]:
# Ensure unique
a = len(song_to_abbrv)
b = len(list(set(song_to_abbrv.values())))
print(a,b)
count = pd.Series(list(song_to_abbrv.values())).value_counts().to_dict()
[x for x in count if count[x] > 1]
assert a == b

965 965


In [18]:
live_songs = pd.DataFrame(songs)

songs = (pd.DataFrame({'code' :song_to_abbrv, 'artist' : artist})
         .reset_index().rename(columns={'index' : 'name'}))

live_songs = (live_songs.merge(songs, on=['name', 'artist'])
              .drop(columns=['name', 'artist']).rename(columns={'code' : 'song'}))
live_songs = live_songs[['show', 'set', 'song', 'order', 'parent', 'out_transition', 
                         'tag', 'stewart', 'stewart_with_lyrics']]

songs = songs.rename(columns={'code' : 'primary_key'}).set_index('primary_key')

### Add Hall Of Fame Column

In [19]:
hof_songs_list = []
f = open("data/hall_of_fame.txt", "r")
hof_songs = [x for x in f]
cutoffs = [i for i in range(len(hof_songs)) if hof_songs[i] == '\n']
for i in range(1, len(cutoffs)):
    year_songs = hof_songs[cutoffs[i-1]+1:cutoffs[i]]
    for s in year_songs:
        if i == 1:
            name = s.split('\t')[0]
            date = s.split('\t')[2].replace('.','-')
        elif i in [2,3,4,6,7,8,9,10]:
            name = s.split('(')[0].rstrip('> ')
            date = s.split('(')[1].rstrip(')').split(' ')[0].replace('/','-')
        elif i == 5:
            name = s.split('|')[0].split('(')[0].strip('> ')
            date = s.split('|')[0].split('(')[1].split(')')[-1].strip().replace('/','-')
        else:
            'issue'
            
        # Manual change
        if name == 'Got Your Milk':
            name = 'Got Your Milk (Right Here)'
            
        song = songs[songs.name == name].index[0]
        date = date.strip('\n').replace('.','-').strip('()')
        if i != 1:
            m,d,y = date.split('-')
            m = m.zfill(2)
            d = d.zfill(2)
            y = int(y) + 2000
            date = '-'.join([m,d,str(y)])
        show = list(shows[(shows.date == date) & ~(shows.vip)].index)
        assert len(show) > 0
        
        # Manual change
        if song == 'HAUN' and date == '11-05-2011':
            show = ['11-05-2011-Milwaukee-3-19']
        
        hof_songs_list.append({'song' : song, 'shows' : show})

In [20]:
live_songs['hof'] = False
for hof_song in hof_songs_list:
    for hof_show in hof_song['shows']:
        assert len(live_songs[(live_songs.song == hof_song['song']) & 
                              (live_songs.show == hof_show)]) > 0
        live_songs.at[(live_songs.song == hof_song['song']) & 
                      (live_songs.show == hof_show), 'hof'] = True

### More Data Validation

In [21]:
for index, row in shows.iterrows():
    set_ct = row['set_ct']
    if set_ct == -1:
        num = len(live_songs[live_songs.show == index])
        assert num == 0
    else:
        na_transitions = sum(live_songs[live_songs.show == index]['out_transition'].isna())
        assert set_ct == na_transitions

### Preview

In [22]:
display(shows.head())
display(songs.head())
display(live_songs.head())

,date,title,venue,city,state,country,vip,support,notes,tags,set_ct,song_ct
primary_key,,,,,,,,,,,,
01-17-2020-New_York-3-18,01-17-2020,"Beacon Theatre, New York, NY, USA",Beacon Theatre,New York,NY,USA,False,None,[Anthem was in remembrance of Neil Peart],{1: 'began with More Than Words (Extreme) teas...,3,18
01-18-2020-New_York-3-15,01-18-2020,"Beacon Theatre, New York, NY, USA",Beacon Theatre,New York,NY,USA,False,None,[Limelight was in remembrance of Neil Peart],"{1: 'with Bright Lights, Big City jam', 2: 'Ai...",3,15
01-19-2020-Brooklyn-3-20,01-19-2020,"Brooklyn Bowl, Brooklyn, NY, USA",Brooklyn Bowl,Brooklyn,NY,USA,False,None,None,"{1: 'began with YYZ (Rush) tease', 2: 'with Q*...",3,20
01-23-2020-Jim_Thorpe-3-13,01-23-2020,"Penn's Peak, Jim Thorpe, PA, USA",Penn's Peak,Jim Thorpe,PA,USA,False,The New Deal,None,"{1: 'with Rhiannon (Fleetwood Mac) teases', 2:...",3,13
01-24-2020-Pittsburgh-3-18,01-24-2020,"Stage AE, Pittsburgh, PA, USA",Stage AE,Pittsburgh,PA,USA,False,The New Deal,[last Tom Sawyer 02.18.2017 (264 shows)],"{1: 'debut, original; with Jake on acoustic', ...",3,18


,name,artist
primary_key,,
BJSR,"""Brendan & Jake switch rigs""",Umphrey's McGee
STRU,"""Mrs Robinson's Strut""",Umphrey's McGee
N19,#19,Umphrey's McGee
N5,#5,Umphrey's McGee
reap,(Don't Fear) The Reaper,Blue Öyster Cult


,show,set,song,order,parent,out_transition,tag,stewart,stewart_with_lyrics,hof
0,01-17-2020-New_York-3-18,Set 1,HERO,1,True,>,-1,False,False,False
1,05-25-2019-Chillicothe-1-7,Set 1,HERO,1,True,>,1,False,False,False
2,06-22-2019-Morrison-3-17,Set 1,HERO,1,True,>,-1,False,False,False
3,07-20-2019-Scranton-1-11,One Set,HERO,1,True,>,-1,False,False,False
4,08-30-2019-Asbury_Park-3-18,Set 1,HERO,1,True,>,-1,False,False,False


### Export

In [23]:
shows.to_pickle('pulls/pull_%s/shows_%s.pickle' % (date_stamp(),date_stamp()))
songs.to_pickle('pulls/pull_%s/songs_%s.pickle' % (date_stamp(),date_stamp()))
live_songs.to_pickle('pulls/pull_%s/live_songs_%s.pickle' % (date_stamp(),date_stamp()))

### Export Song Abbreviations

In [24]:
code_to_name = songs['name'].to_dict()
name_to_code = {code_to_name[i] : i for i in code_to_name}
codes = list(collections.OrderedDict(sorted(name_to_code.items())).items())

In [25]:
table = {}
for i in range(100):
    table[i] = []
for i in range(len(codes)):
    table[i % 100].append(codes[i])
for i in range(100):
    table[i] = [item for sublist in table[i] for item in sublist]
table = list(table.values())

In [26]:
pd.DataFrame(table).to_csv('data/song_codes.csv')